In [38]:
### Import the libraries we are going to use;

import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import os
import xgboost
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [39]:
### Set the right directory;
os.chdir(r"C:\Users\cli\Documents\Work\Email Marketing\Email_Open_Segmentation")
os.getcwd()

'C:\\Users\\cli\\Documents\\Work\\Email Marketing\\Email_Open_Segmentation'

In [40]:
### Read csv file
Trial_day0_Open = pd.read_csv("Email_Opens_0322.csv")
Trial_day0_Open = Trial_day0_Open.iloc[:, 1:]
Trial_day0_Open.head()

,canceled_at,card_type,channel,created_at,device,email_domain,intent,is_ios,is_us,open_time,product,quality,quota,sent_time,state,status,type,user_id,user_id_intent,user_id_quota
0,8/11/2019 3:50:14 PM,Visa,SEM & GDN,8/3/2019 12:51:09 AM,Desktop,gmail.com,other,1,0,8/3/2019 8:32:11 PM,address,NaN,2.0,8/3/2019 2:45:35 AM,Connecticut,0,DEBIT,110542746,110542746,110542746.0
1,10/22/2019 7:48:18 AM,Visa,Affiliate & Others,8/3/2019 12:51:26 AM,Mobile,cox.net,scams_fraud,1,0,8/4/2019 3:12:45 AM,phone,NANPA,1.0,8/3/2019 2:45:34 AM,Virginia,0,DEBIT,110542754,110542754,110542754.0
2,10/10/2019 12:59:47 PM,MasterCard,SEO & Direct,8/3/2019 12:52:05 AM,Mobile,gmail.com,screen_dates,1,0,8/3/2019 3:15:18 AM,name,uncredible,1.0,8/3/2019 2:45:35 AM,Georgia,0,DEBIT,110542762,110542762,110542762.0
3,8/4/2019 7:11:23 PM,Amex,SEM & GDN,8/3/2019 12:52:53 AM,Mobile,hotmail.com,contacts,0,0,8/3/2019 3:48:14 AM,phone,1 PID,1.0,8/3/2019 2:45:35 AM,California,0,DEBIT,110542772,110542772,110542772.0
4,10/2/2019 12:55:26 PM,MasterCard,SEM & GDN,8/3/2019 12:59:21 AM,Mobile,yahoo.com,scams_fraud,1,0,8/3/2019 2:47:29 AM,name,high,5.0,8/3/2019 2:45:34 AM,Indiana,0,CREDIT,110542816,110542816,110542816.0


In [41]:
## Drop some columns;

tdo = Trial_day0_Open.drop(['canceled_at', 'created_at', 'sent_time', 'open_time', 'user_id', 'user_id_intent', 'user_id_quota'], axis = 1)
tdo.head()

,card_type,channel,device,email_domain,intent,is_ios,is_us,product,quality,quota,state,status,type
0,Visa,SEM & GDN,Desktop,gmail.com,other,1,0,address,NaN,2.0,Connecticut,0,DEBIT
1,Visa,Affiliate & Others,Mobile,cox.net,scams_fraud,1,0,phone,NANPA,1.0,Virginia,0,DEBIT
2,MasterCard,SEO & Direct,Mobile,gmail.com,screen_dates,1,0,name,uncredible,1.0,Georgia,0,DEBIT
3,Amex,SEM & GDN,Mobile,hotmail.com,contacts,0,0,phone,1 PID,1.0,California,0,DEBIT
4,MasterCard,SEM & GDN,Mobile,yahoo.com,scams_fraud,1,0,name,high,5.0,Indiana,0,CREDIT


In [42]:
### Convert column 'quality'; 
tdo.quality.unique()

array([nan, 'NANPA', 'uncredible', '1 PID', 'high', 'Third_Party',
       'More than 1 PID', 'medium', 'Similar Result', 'List View',
       'Social Profile View', 'Telescope', 'overmerged', 'low',
       'No Result', 'sparse'], dtype=object)

In [43]:
## Change the quality into a new grouping; 

tdo['quality'] = tdo.quality.astype(str)
# Others
tdo['p_quality'] = 'Medium'

# Phone 
tdo['p_quality'][(tdo['quality'] == '1 PID') | (tdo['quality'] == 'More than 1 PID')] = 'High'
tdo['p_quality'][(tdo['quality'] == 'Third_Party') | (tdo['quality'] == 'Telescope')] = 'Medium'
tdo['p_quality'][tdo['quality'] == 'NANPA'] = 'Low'

# Name
tdo['p_quality'][(tdo['quality'] == 'high') | (tdo['quality'] == 'overmerged')] = 'High'
tdo['p_quality'][(tdo['quality'] == 'uncredible') | (tdo['quality'] == 'medium')] = 'Medium'
tdo['p_quality'][(tdo['quality'] == 'low') | (tdo['quality'] == 'sparse')] = 'Low'

# Email
tdo['p_quality'][(tdo['quality'] == 'List View') | (tdo['quality'] == 'Social Profile View')] = 'High'
tdo['p_quality'][tdo['quality'] == 'Similar Result'] = 'Medium'
tdo['p_quality'][tdo['quality'] == 'No Result'] = 'Low'



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13

In [44]:
## Convert some fields - 
tdo['state'] = tdo.state.astype(str)

tdo['email'] = 'Others'
tdo['email'][tdo['email_domain'] == 'gmail.com'] = 'Google'
tdo['email'][tdo['email_domain'] == 'comcast.net'] = 'Comcast'
tdo['email'][tdo['email_domain'] == 'iCloud.com'] = 'Apple'
tdo['email'][(tdo['email_domain'] == 'yahoo.com') | (tdo['email_domain'] == 'aol.com') | (tdo['email_domain'] == 'att.net')] = 'Verizon'
tdo['email'][(tdo['email_domain'] == 'hotmail.com') | (tdo['email_domain'] == 'outlook.com') | (tdo['email_domain'] == 'msn.com') | (tdo['email_domain'] == 'live.com')] = 'Microsoft'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [9]:
tdo.head()


,card_type,channel,device,email_domain,intent,is_ios,is_us,product,quality,quota,state,status,type,p_quality,email
0,Visa,SEO & Direct,Mobile,yahoo.com,friends_family,1,0,phone,NANPA,1.0,Nevada,0,DEBIT,Low,Verizon
1,Visa,Affiliate & Others,Mobile,icloud.com,unknown_phone,1,0,phone,NANPA,1.0,Arizona,0,DEBIT,Low,Others
2,Visa,SEO & Direct,Mobile,gmail.com,screen_dates,1,0,name,overmerged,1.0,Washington,0,DEBIT,High,Google
3,Visa,SEM & GDN,Mobile,gmail.com,other,0,0,phone,Third_Party,1.0,Washington,0,DEBIT,Medium,Google
4,Visa,Affiliate & Others,Mobile,gmail.com,unknown_phone,0,0,phone,Third_Party,1.0,New York,0,DEBIT,Medium,Google


In [45]:
tdo.type.unique()

array(['DEBIT', 'CREDIT', 'CHARGE CARD', nan], dtype=object)

In [47]:
tdo.p_quality.unique()

array(['Medium', 'Low', 'High'], dtype=object)

In [48]:
### Try One-Hot encoding; 

## Part 1 - Card Type
card_type = ('Visa', 'MasterCard', 'Amex', 'Discover')
card_df = pd.DataFrame(card_type, columns=['card_type'])
dum_df = pd.get_dummies(card_df, columns=["card_type"], prefix=["Is"] )

card_df = card_df.join(dum_df)
card_df



,card_type,Is_Amex,Is_Discover,Is_MasterCard,Is_Visa
0,Visa,0,0,0,1
1,MasterCard,0,0,1,0
2,Amex,1,0,0,0
3,Discover,0,1,0,0


In [49]:
## Part 2 - Channel 
channel = ('SEO & Direct', 'SEM & GDN', 'Affiliate & Others')
channel_df = pd.DataFrame(channel, columns=['channel'])
dum_df_1 = pd.get_dummies(channel_df, columns=["channel"], prefix=["Is"] )

channel_df = channel_df.join(dum_df_1)
channel_df

,channel,Is_Affiliate & Others,Is_SEM & GDN,Is_SEO & Direct
0,SEO & Direct,0,0,1
1,SEM & GDN,0,1,0
2,Affiliate & Others,1,0,0


In [50]:
## Part 3 - Device

device = ('Mobile', 'Desktop')
device_df = pd.DataFrame(device, columns=['device'])
dum_df_2 = pd.get_dummies(device_df, columns=["device"], prefix=["Is"] )

device_df = device_df.join(dum_df_2)
device_df

,device,Is_Desktop,Is_Mobile
0,Mobile,0,1
1,Desktop,1,0


In [51]:
## Part 4 - Email

email = ('Others', 'Google', 'Comcast', 'Apple', 'Microsoft', 'Verizon')
email_df = pd.DataFrame(email, columns=['email'])
dum_df_3 = pd.get_dummies(email_df, columns=["email"], prefix=["Is"] )

email_df = email_df.join(dum_df_3)
email_df

,email,Is_Apple,Is_Comcast,Is_Google,Is_Microsoft,Is_Others,Is_Verizon
0,Others,0,0,0,0,1,0
1,Google,0,0,1,0,0,0
2,Comcast,0,1,0,0,0,0
3,Apple,1,0,0,0,0,0
4,Microsoft,0,0,0,1,0,0
5,Verizon,0,0,0,0,0,1


In [52]:
## Part 5 - Intent;

intent = ('friends_family', 'unknown_phone', 'screen_dates', 'other',
       'scams_fraud', 'infidelity', 'self', 'contacts')
intent_df = pd.DataFrame(intent, columns=['intent'])
dum_df_4 = pd.get_dummies(intent_df, columns=["intent"], prefix=["Is"] )

intent_df = intent_df.join(dum_df_4)
intent_df

,intent,Is_contacts,Is_friends_family,Is_infidelity,Is_other,Is_scams_fraud,Is_screen_dates,Is_self,Is_unknown_phone
0,friends_family,0,1,0,0,0,0,0,0
1,unknown_phone,0,0,0,0,0,0,0,1
2,screen_dates,0,0,0,0,0,1,0,0
3,other,0,0,0,1,0,0,0,0
4,scams_fraud,0,0,0,0,1,0,0,0
5,infidelity,0,0,1,0,0,0,0,0
6,self,0,0,0,0,0,0,1,0
7,contacts,1,0,0,0,0,0,0,0


In [53]:
## Part 6 - Product
product = ('name', 'phone', 'email', 'address')
product_df = pd.DataFrame(product, columns=['product'])
dum_df_5 = pd.get_dummies(product_df, columns=["product"], prefix=["Is"] )

product_df = product_df.join(dum_df_5)
product_df


,product,Is_address,Is_email,Is_name,Is_phone
0,name,0,0,1,0
1,phone,0,0,0,1
2,email,0,1,0,0
3,address,1,0,0,0


In [54]:
## Part 7 - State
state = ('Nevada', 'Arizona', 'Washington', 'New York', 'California',
       'Wisconsin', 'Texas', 'Kentucky', 'Missouri', 'Florida',
       'Pennsylvania', 'Michigan', 'Virginia', 'Georgia', 'New Jersey',
       'Delaware', 'Connecticut', 'Oklahoma', 'Colorado', 'Oregon',
       'Alabama', 'Indiana', 'Massachusetts', 'Nebraska', 'Maryland',
       'West Virginia', 'Illinois', 'Mississippi', 'South Carolina',
       'Louisiana', 'Ohio', 'District of Columbia', 'Iowa', 'Minnesota',
       'North Carolina', 'Tennessee', 'Arkansas', '0', 'Alaska', 'Idaho',
       'Rhode Island', 'New Hampshire', 'Utah', 'Wyoming', 'Kansas',
       'Vermont', 'New Mexico', 'nan', 'South Dakota', 'Hawaii',
       'Montana', 'North Dakota', 'Maine')
state_df = pd.DataFrame(state, columns=['state'])
dum_df_6 = pd.get_dummies(state_df, columns=["state"], prefix=["Is"] )

state_df = state_df.join(dum_df_6)



In [55]:
## Part 8 - P_Quality
p_quality = ('Low', 'Medium', 'High')
p_quality_df = pd.DataFrame(p_quality, columns=['p_quality'])
dum_df_7 = pd.get_dummies(p_quality_df, columns=["p_quality"], prefix=["Is"] )

p_quality_df = p_quality_df.join(dum_df_7)
p_quality_df


,p_quality,Is_High,Is_Low,Is_Medium
0,Low,0,1,0
1,Medium,0,0,1
2,High,1,0,0


In [56]:
tdo_01 = pd.merge(tdo,
                  card_df,
                  on='card_type')


tdo_01 = pd.merge(tdo_01,
                  channel_df,
                  on='channel')

tdo_01 = pd.merge(tdo_01,
                  device_df,
                  on='device')

tdo_01 = pd.merge(tdo_01,
                  email_df,
                  on='email')

tdo_01 = pd.merge(tdo_01,
                  intent_df,
                  on='intent')

tdo_01 = pd.merge(tdo_01,
                  product_df,
                  on='product')

tdo_01 = pd.merge(tdo_01,
                  state_df,
                  on='state')

tdo_01 = pd.merge(tdo_01,
                  p_quality_df,
                  on='p_quality')

tdo_01.head()

,card_type,channel,device,email_domain,intent,is_ios,is_us,product,quality,quota,...,Is_Vermont,Is_Virginia,Is_Washington,Is_West Virginia,Is_Wisconsin,Is_Wyoming,Is_nan,Is_High,Is_Low,Is_Medium
0,Visa,SEM & GDN,Desktop,gmail.com,other,1,0,address,nan,2.0,...,0,0,0,0,0,0,0,0,0,1
1,MasterCard,SEM & GDN,Mobile,gmail.com,other,1,0,address,nan,10.0,...,0,0,0,0,0,0,0,0,0,1
2,Visa,SEO & Direct,Mobile,gmail.com,other,0,0,address,nan,3.0,...,0,0,0,0,0,0,0,0,0,1
3,Visa,SEO & Direct,Mobile,gmail.com,other,0,0,address,nan,2.0,...,0,0,0,0,0,0,0,0,0,1
4,Visa,SEO & Direct,Mobile,gmail.com,other,0,0,address,nan,4.0,...,0,0,0,0,0,0,0,0,0,1


In [57]:
tdo_01.head()


,card_type,channel,device,email_domain,intent,is_ios,is_us,product,quality,quota,...,Is_Vermont,Is_Virginia,Is_Washington,Is_West Virginia,Is_Wisconsin,Is_Wyoming,Is_nan,Is_High,Is_Low,Is_Medium
0,Visa,SEM & GDN,Desktop,gmail.com,other,1,0,address,nan,2.0,...,0,0,0,0,0,0,0,0,0,1
1,MasterCard,SEM & GDN,Mobile,gmail.com,other,1,0,address,nan,10.0,...,0,0,0,0,0,0,0,0,0,1
2,Visa,SEO & Direct,Mobile,gmail.com,other,0,0,address,nan,3.0,...,0,0,0,0,0,0,0,0,0,1
3,Visa,SEO & Direct,Mobile,gmail.com,other,0,0,address,nan,2.0,...,0,0,0,0,0,0,0,0,0,1
4,Visa,SEO & Direct,Mobile,gmail.com,other,0,0,address,nan,4.0,...,0,0,0,0,0,0,0,0,0,1


In [58]:
## Drop some features;

tdo_02 = tdo_01.drop(['card_type', 'channel', 'device', 'email_domain', 'intent', 'is_us', 'product', 'quality', 'state', 'type', 'email', 'p_quality'], axis = 1)
tdo_02.head()

,is_ios,quota,status,Is_Amex,Is_Discover,Is_MasterCard,Is_Visa,Is_Affiliate & Others,Is_SEM & GDN,Is_SEO & Direct,...,Is_Vermont,Is_Virginia,Is_Washington,Is_West Virginia,Is_Wisconsin,Is_Wyoming,Is_nan,Is_High,Is_Low,Is_Medium
0,1,2.0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1,10.0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,3.0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0,2.0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0,4.0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [59]:
tdo_02.dtypes['quota']
tdo_02['quota'].clip_upper(20)
tdo_02["quota"] = tdo_02["quota"].fillna(0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: clip_upper(threshold) is deprecated, use clip(upper=threshold) instead
  


In [60]:
tdo_02.status.value_counts()

0    169981
1     21987
Name: status, dtype: int64

In [ ]:
## Data Imbalancing exists, we want to try not upsample / downsample first; 

In [61]:
cols = ['status']
y = tdo_02['status']
X = tdo_02[tdo_02.columns.difference(['status'])]

In [28]:
## Part b, try Random Forest;
clf = RandomForestClassifier(n_estimators = 100, random_state = 456, criterion = 'entropy', max_depth = 70)
clf.fit(X_train,y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=70, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=456,
                       verbose=0, warm_start=False)

In [29]:
## Performance
y_pred_1=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_1))

Accuracy: 0.8707784202392735


In [30]:
## Another look
print(classification_report(y_test, y_pred_1))

              precision    recall  f1-score   support

           0       0.89      0.97      0.93     51103
           1       0.21      0.05      0.08      6488

    accuracy                           0.87     57591
   macro avg       0.55      0.51      0.51     57591
weighted avg       0.81      0.87      0.83     57591



In [31]:
## Feature Importance
feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

feature_importances[:20]

,importance
quota,0.253757
is_ios,0.050639
Is_Medium,0.026489
Is_High,0.026478
Is_California,0.024768
Is_Visa,0.023570
Is_Low,0.022833
Is_MasterCard,0.022299
Is_contacts,0.020703
Is_friends_family,0.019438


In [62]:
from sklearn.utils import resample

# Separate majority and minority classes
tdo_02_majority = tdo_02[tdo_02.status==0]
tdo_02_minority = tdo_02[tdo_02.status==1]

tdo_02_minority_upsampled = resample(tdo_02_minority, 
                                 replace=True,     # sample with replacement        
                                 n_samples=169981,    # to match majority class
                                 random_state=123) # reproducible results

# Combine majority class with upsampled minority class
tdo_02_upsampled = pd.concat([tdo_02_minority_upsampled, tdo_02_majority])

tdo_02_upsampled.status.value_counts()

1    169981
0    169981
Name: status, dtype: int64

In [63]:
cols = ['status']
y0 = tdo_02_upsampled['status']
X0 = tdo_02_upsampled[tdo_02_upsampled.columns.difference(['status'])]

In [64]:
### Part b, Random Forest
X1_train, X1_test, y1_train, y1_test = train_test_split(X0, y0, test_size=0.3, random_state = 100)
clf1 = RandomForestClassifier(n_estimators = 200, random_state = 456, criterion = 'entropy', max_depth = 70)
clf1.fit(X1_train,y1_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=70, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=456,
                       verbose=0, warm_start=False)

In [65]:
y1_pred=clf1.predict(X1_test)
print("Accuracy:",metrics.accuracy_score(y1_test, y1_pred))

Accuracy: 0.8622008255792292


In [66]:
print(classification_report(y1_test, y1_pred))

              precision    recall  f1-score   support

           0       0.91      0.80      0.85     50932
           1       0.82      0.92      0.87     51057

    accuracy                           0.86    101989
   macro avg       0.87      0.86      0.86    101989
weighted avg       0.87      0.86      0.86    101989



In [67]:
feature_importances_1 = pd.DataFrame(clf1.feature_importances_,
                                     index = X1_train.columns,
                                     columns=['importance']).sort_values('importance', ascending=False)
feature_importances_1[:20]

,importance
quota,0.245075
is_ios,0.055451
Is_Medium,0.026027
Is_California,0.025208
Is_High,0.024812
Is_Low,0.021604
Is_Visa,0.020828
Is_Texas,0.020200
Is_contacts,0.020166
Is_MasterCard,0.019501


In [68]:
sum(feature_importances_1[:20]['importance'])

0.6363761062979106

In [69]:
## Try to obtain percentage using 'is_ios' column;

tdo_02.groupby('status').mean()[['quota', 'is_ios', 'Is_Medium', 'Is_High', 'Is_California', 'Is_Low', 'Is_Visa', 'Is_Texas', 'Is_contacts', 'Is_MasterCard', 'Is_friends_family', 'Is_scams_fraud', 'Is_SEM & GDN', 'Is_Washington', 'Is_self', 'Is_Arizona', 'Is_Affiliate & Others', 'Is_infidelity', 'Is_name', 'Is_New York']]

,quota,is_ios,Is_Medium,Is_High,Is_California,Is_Low,Is_Visa,Is_Texas,Is_contacts,Is_MasterCard,Is_friends_family,Is_scams_fraud,Is_SEM & GDN,Is_Washington,Is_self,Is_Arizona,Is_Affiliate & Others,Is_infidelity,Is_name,Is_New York
status,,,,,,,,,,,,,,,,,,,,
0,3.183991,0.624870,0.371995,0.400121,0.208070,0.227884,0.654450,0.112413,0.227737,0.296027,0.151935,0.127155,0.634594,0.093375,0.099923,0.117348,0.146858,0.110524,0.283373,0.042681
1,3.072816,0.624869,0.374312,0.391231,0.204803,0.234457,0.667986,0.109246,0.241006,0.282030,0.148906,0.129031,0.621777,0.091645,0.091418,0.109656,0.139037,0.100241,0.278392,0.042207


In [70]:
tdo_02.groupby('quota').mean()[['status']]

,status
quota,
0.0,0.104046
1.0,0.121228
2.0,0.116111
3.0,0.113992
4.0,0.110868
5.0,0.108743
6.0,0.111721
7.0,0.107794
8.0,0.108739


In [75]:
## Add one more variable based on quota;
row_indexes=tdo_02[tdo_02['quota'] >= 4].index
tdo_02.loc[row_indexes,'4 or more'] = "1"

row_indexes_1=tdo_02[tdo_02['quota'] < 4].index
tdo_02.loc[row_indexes_1,'4 or more'] = "0"

In [76]:
## Obtain group count;
Pct_1 = tdo_02.groupby(['4 or more', 'is_ios','Is_High'])['status'].count().reset_index()

Pct_1['Percentage'] = round(100 * Pct_1['status']  / Pct_1['status'].sum(), 2)

print(Pct_1)

  4 or more  is_ios  Is_High  status  Percentage
0         0       0        0   29042       15.13
1         0       0        1   17546        9.14
2         0       1        0   49718       25.90
3         0       1        1   33458       17.43
4         1       0        0   15171        7.90
5         1       0        1   10254        5.34
6         1       1        0   21422       11.16
7         1       1        1   15357        8.00


In [77]:
## Obtain group level cancel rate;
Pct_2 = tdo_02.groupby(['4 or more', 'is_ios','Is_High','status'])['Is_California'].count().reset_index()

Pct_2

,4 or more,is_ios,Is_High,status,Is_California
0,0,0,0,0,25702
1,0,0,0,1,3340
2,0,0,1,0,15487
3,0,0,1,1,2059
4,0,1,0,0,43690
5,0,1,0,1,6028
6,0,1,1,0,29658
7,0,1,1,1,3800
8,1,0,0,0,13512
9,1,0,0,1,1659
